In [29]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar 11 15:55:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [30]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U transformers
# Install below if using GPU
!pip install accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('hf_token')

login(hf_token)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Example Code for Running Gemma 2B Model

In [43]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(outputs[0]))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

Machines, they weave and they learn,
From


In [ ]:
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
import vertexai

REGION = "us-central1"

PROJECT_ID = "airy-timing-354405"
# Set the project id
vertexai.init(project=PROJECT_ID, location=REGION)

# A function to generate text embedding.
def text_embedding(
  model_name: str, task_type: str, text: str, title: str = "") -> list:
    """Generate text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained(model_name)

    text_embedding_input = TextEmbeddingInput(
        task_type=task_type, title=title, text=text)
    embeddings = model.get_embeddings([text_embedding_input])
    return embeddings[0].values

# Set the model name.
MODEL_NAME = "textembedding-gecko@002"


TASK_TYPE = "RETRIEVAL_DOCUMENT"
TITLE = "Google"
TEXT = "Embed text."

embedding = text_embedding(model_name=MODEL_NAME, task_type=TASK_TYPE, text=TEXT, title=TITLE)
print(len(embedding))


In [ ]:
!pip install langchain singlestoredb tiktoken openai pypdf
!pip install langchain_openai

In [42]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import SingleStoreDB
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
import os

loader = PyPDFLoader("superbowl.pdf")
data = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
key = userdata.get('openai')
# Generate embeddings
embeddings = OpenAIEmbeddings(openai_api_key=key)

# Set up the SingleStoreDB connection and vector database
os.environ[
    "SINGLESTOREDB_URL"] = "ayush:Test1234@svc-3482219c-a389-4079-b18b-d50662524e8a-shared-dml.aws-virginia-6.svc.singlestore.com:3333/database_79fb0"
SingleStoreVectorDB = SingleStoreDB.from_documents(texts, embeddings, table_name="notebook")


In [46]:

SingleStoreVectorDB = SingleStoreDB.from_documents(texts, embeddings, table_name="notebook")

user_query = "Who won the superbowl"

# Perform similarity search
docs = SingleStoreVectorDB.similarity_search(user_query)
if docs:
    context = docs[0].page_content

    from transformers import AutoTokenizer, AutoModelForCausalLM

    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
    model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

    combined_information = "User Query: " +  user_query + "Context: " + context

    # Moving tensors to GPU
    input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
    response = model.generate(**input_ids, max_new_tokens=500)
    print(tokenizer.decode(response[0]))


else:
    print("AI: Sorry, I couldn't find relevant information.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<bos>User Query: Who won the superbowlContext: Super Bowl LVIII was an American football game played to determine the champion of the National Football League (NFL) for the 2023 season. In a rematch of Super Bowl LIV from four years earlier, the American Football Conference (AFC) champion and defending Super Bowl champion Kansas City Chiefs defeated the National Football Conference (NFC) champion San Francisco 49ers 25–22 in overtime. The Chiefs became the first team to win back-to-back Super Bowls since the New England Patriots in 2004.[6] The game was played on February 11, 2024, at Allegiant Stadium in Paradise, Nevada. This was the first Super Bowl to be held in the state of Nevada.[7][8] It marked the third straight year that the Super Bowl had been played in the Western United States, following host cities Inglewood, California, in 2022 and Glendale, Arizona, in 2023.

What was the outcome of the Super Bowl LVIII game?

A. Kansas City Chiefs defeated San Francisco 49ers 25–22 in 